**Input**
- MAGMA gene analysis results for various GWAS
   - Chronotype (1180)
   - ACCEL results
       - 28450_ST_long_mean
       - 28451_ST_long_sd
       - 28452_WT_long_mean
       - 28453_WT_long_sd
       - 28454_ST_short_mean
       - 28455_ST_short_sd
       - 28456_WT_short_mean
       - 28457_WT_short_sd
       - 28458_long_window_len_mean
       - 28459_long_window_len_sd
       - 28460_long_window_num_mean
       - 28461_long_window_num_sd
       - 28462_short_window_len_mean
       - 28463_short_window_len_sd
       - 28464_short_window_num_mean
       - 28465_short_window_num_sd
       - 28466_phase_mean
       - 28467_phase_sd
       - 28468_max_period
       - 28469_amplitude
       - 28470_sleep_percentage

**Process: calculate the following**
- MAGMA
  - Adding info
    - Bonferroni-corrected P-values (also written P-Bonf)
    - q-values (also known as FDR-adjusted p-values)
    - Annotation of whether those values are under FDR threshold


# Preparation (Execute all in this section!)

## Import libraries & set environment variables

In [1]:
import collections
from datetime import datetime
import os
import numpy as np
from pathlib import Path
import polars as pl
import re
import textwrap

import warnings
# suppress DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

from matplotlib import pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

dir_home = Path(os.getcwd()).parent.parent.parent
os.chdir(dir_home)
print("Current directory (check that it's your home directory):", os.getcwd())

Current directory (check that it's your home directory): J:\sugai\UKBiobank


## File I/O

In [2]:
# Input
DIR_SOURCE = os.path.join("analysis", "specific", "analysis_20230526_01_gwas")

# Gene alias list
FILE_ALIAS=os.path.join("data", "refseq", "gene_full_name_tab.txt")
df_alias = pl.read_csv(FILE_ALIAS, separator="\t")

# Output folder
DIR_OUT = os.path.join(DIR_SOURCE, "summary")
if not os.path.exists(DIR_OUT):
    os.makedirs(DIR_OUT)

# # Intermediate output: added info
# #     (q-values and statistically significant genes)
# FILE_ANNOTATED_PANUKBB = os.path.join(DIR_OUT, "out.with_p_correction_PanUKBB.csv")
# FILE_ANNOTATED_MYGWAS_1 = os.path.join(DIR_OUT, "out.with_p_correction_myGWAS_1.csv")
# FILE_ANNOTATED_MYGWAS_2 = os.path.join(DIR_OUT, "out.with_p_correction_myGWAS_2.csv")

FDR_THRESHOLD = 0.05 # Q value threshold

## Functions

In [6]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.abc.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el


def func_print(i):
    if i < 10 or str(i)[1:].count('0') == len(str(i))-1:
        now = datetime.now()
        print(f"{now.strftime('%Y-%m-%d %H:%M:%S')}: {i}")
        

def func_process(FILE_SOURCE, FILE_OUTPUT):
    print("Processing:", FILE_SOURCE)
    # Read as String to avoid any automatic conversion
    df = (pl.read_csv(FILE_SOURCE, 
                      separator=",", 
                      infer_schema_length=0)
          .drop_nulls()
          .with_columns(pl.col("P").cast(pl.Float32).alias("P_Bonf"))
          .with_columns(pl.col("P").cast(pl.Float32).alias("P_log10"))
          .with_columns(pl.col("P").cast(pl.Float32).alias("Q"))
          .with_columns(np.log10(pl.col("P_log10")))
          .sort(by='Q'))

    n_count = df['Q'].is_not_null().sum()

    df = (df
          # Calculate Bonferroni-corrected P values
          .with_columns(pl.col('P_Bonf') * n_count)
           # create a new column which is true if "P_Bonf" is under threshold
          .with_columns((pl.col("P_Bonf") < FDR_THRESHOLD).alias("Positive_P_Bonf"))
          # Calculate Q values
          # Multiply Q column by number of non-null values
          .with_columns(pl.col('Q') * n_count)
          # # Divide the Q columns by the rank of individual value among all the non-null values
          .with_columns(pl.col('Q') / df['Q'].rank())
          # create a new column which is true if "q" is under threshold
          .with_columns((pl.col("Q") < FDR_THRESHOLD).alias("Positive_Q"))
          # Add log10 values
          .with_columns(pl.col("P_Bonf").alias("P_Bonf_log10"))
          .with_columns(np.log10(pl.col("P_Bonf_log10")))
          .with_columns(pl.col("Q").alias("Q_log10"))
          .with_columns(np.log10(pl.col("Q_log10")))
          # Sort
          .sort(by='Positive_P_Bonf', descending=True)
         )
    df.write_csv(FILE_OUTPUT)
    
    
def func_list_genes(FILE_SOURCE):
    df_all = pl.read_csv(FILE_SOURCE, separator=",")
    df_positive = df_all.filter(pl.col("Positive_P_Bonf"))
    set_genes_all = set(df_all["Gene"])
    set_genes_positive = set(df_positive["Gene"])
    print("All =", len(set_genes_all), "genes, positive =", len(set_genes_positive), "genes")
    return set_genes_all, set_genes_positive

# Process MAGMA result statistics

## Add Pbonf, q-values, posi/nega annotation

In [4]:
list_files = []
for root, dirs, files in os.walk(DIR_SOURCE):
    # Check if the target file exists in the current folder
    if "gene_pheno_pval_list_annotated.csv" in files:
        # Get the full path of the file
        file_source = os.path.join(root, "gene_pheno_pval_list_annotated.csv")
        file_out = os.path.join(root, "with_p_correction.csv")
        # Print the file path
        list_files.append([file_source, file_out])

print(len(list_files), "Files are found")
list_files

22 Files are found


[['analysis\\specific\\analysis_20230526_01_gwas\\28461_long_window_num_sd\\magma\\gene_pheno_pval_list_annotated.csv',
  'analysis\\specific\\analysis_20230526_01_gwas\\28461_long_window_num_sd\\magma\\with_p_correction.csv'],
 ['analysis\\specific\\analysis_20230526_01_gwas\\28469_amplitude\\magma\\gene_pheno_pval_list_annotated.csv',
  'analysis\\specific\\analysis_20230526_01_gwas\\28469_amplitude\\magma\\with_p_correction.csv'],
 ['analysis\\specific\\analysis_20230526_01_gwas\\00901_1180-0.0\\magma\\gene_pheno_pval_list_annotated.csv',
  'analysis\\specific\\analysis_20230526_01_gwas\\00901_1180-0.0\\magma\\with_p_correction.csv'],
 ['analysis\\specific\\analysis_20230526_01_gwas\\28466_phase_mean\\magma\\gene_pheno_pval_list_annotated.csv',
  'analysis\\specific\\analysis_20230526_01_gwas\\28466_phase_mean\\magma\\with_p_correction.csv'],
 ['analysis\\specific\\analysis_20230526_01_gwas\\28468_max_period\\magma\\gene_pheno_pval_list_annotated.csv',
  'analysis\\specific\\analysi

## Extract positive genes

In [7]:
list_sets_genes = []

for list_IO in list_files:
    print()
    func_process(list_IO[0], list_IO[1])
    set_genes_all, set_genes_positive = func_list_genes(list_IO[1])
    list_sets_genes.append([list_IO[1], set_genes_all, set_genes_positive])


Processing: analysis\specific\analysis_20230526_01_gwas\28461_long_window_num_sd\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 0 genes

Processing: analysis\specific\analysis_20230526_01_gwas\28469_amplitude\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 15 genes

Processing: analysis\specific\analysis_20230526_01_gwas\00901_1180-0.0\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 36 genes

Processing: analysis\specific\analysis_20230526_01_gwas\28466_phase_mean\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 3 genes

Processing: analysis\specific\analysis_20230526_01_gwas\28468_max_period\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 1 genes

Processing: analysis\specific\analysis_20230526_01_gwas\28453_WT_long_sd\magma\gene_pheno_pval_list_annotated.csv
All = 17995 genes, positive = 1 genes

Processing: analysis\specific\analysis_20230526_01_gwas\28454_ST_short_mea

In [8]:
# Extract non-empty sets
list_sets_positive_genes = [[l[0], l[2]] for l in list_sets_genes if l[2] != set()]
list_sets_positive_genes = [[l[0].replace("\magma\with_p_correction.csv", ""), l[1]] 
                            for l in list_sets_positive_genes]
list_sets_positive_genes = [[re.sub(".*\\\\", "", l[0]), l[1]] for l in list_sets_positive_genes]

# Concatenate all to one table
for i, l in enumerate(list_sets_positive_genes):
    print(l)
    df = pl.DataFrame({
        "Gene": list(l[1]),
        l[0]: pl.Series([1] * len(l[1]))
    })
    if i == 0:
        df_positive_genes = df.clone()
    else:
        df_positive_genes = df_positive_genes.join(df, on="Gene", how="outer")

        
print()


df_positive_genes = (df_positive_genes
                     .fill_null(0)
                     .sort(by="Gene")
                     .with_columns(
                         pl.fold(0, lambda acc, s: acc + s, pl.all().exclude("Gene"))
                         .alias("positive_count")
                     )
                    )

# Join to alias list
df_positive_genes = df_alias.join(df_positive_genes, on="Gene", how="inner")
print(df_positive_genes.head)
FILE_OUT = os.path.join(DIR_OUT, "positive_genes.csv")
df_positive_genes.write_csv(FILE_OUT)

print("Positive in more than one GWAS:")
df_positive_double = df_positive_genes.filter(pl.col("positive_count") > 1)
print(df_positive_double.head)
FILE_OUT = os.path.join(DIR_OUT, "positive_double.csv")
df_positive_double.write_csv(FILE_OUT)

['28469_amplitude', {'CXXC5', 'ARL17B', 'GLO1', 'BTBD9', 'PCID2', 'PBX3', 'STH', 'KANSL1', 'CRHR1', 'MAPT', 'KAT6A', 'KCNK9', 'LRRC37A', 'MEIS1', 'HOXB3'}]
['00901_1180-0.0', {'ASB1', 'C11orf80', 'CPNE8', 'FTO', 'RBM14', 'CTSF', 'NRXN1', 'PC', 'GPR61', 'RASSF1', 'RGL1', 'RGS16', 'HYAL1', 'RNASEL', 'CLN5', 'PTPN21', 'TTC8', 'FBXL3', 'LSMEM2', 'KHDRBS2', 'LRFN4', 'ROBO2', 'CAMKMT', 'SPATA7', 'VAMP3', 'ALG10B', 'IFRD2', 'KIF21A', 'SOX5', 'ZMYND10', 'RASD1', 'PER3', 'ACTN3', 'SPTBN2', 'RBFOX1', 'FOXP2'}]
['28466_phase_mean', {'RGS16', 'AUTS2', 'ALG10B'}]
['28468_max_period', {'SOX18'}]
['28453_WT_long_sd', {'MEIS1'}]
['28454_ST_short_mean', {'SGCZ'}]
['28452_WT_long_mean', {'PKP4', 'ACP1', 'SH3YL1', 'CCDC148', 'FAM150B', 'SKOR1', 'MEIS1'}]
['28470_sleep_percentage', {'CXXC5', 'ARL17B', 'PROZ', 'ATRNL1', 'BTBD9', 'PCID2', 'PBX3', 'STH', 'CRHR1', 'MAPT', 'LRRC37A', 'MEIS1', 'KANSL1'}]
['28464_short_window_num_mean', {'SMIM4', 'NEK4', 'STAB1', 'PBRM1', 'GNL3', 'SPCS1', 'NT5DC2', 'SALL4', 'GLT